# 基于GNN的金融异常检测任务

## 1. 实验介绍

反欺诈是金融行业永恒的主题，在互联网金融信贷业务中，数字金融反欺诈技术已经得到广泛应用并取得良好效果，这其中包括了近几年迅速发展并在各个领域
得到越来越广泛应用的图神经网络。本项目以互联网智能风控为背景，从用户相互关联和影响的视角，探索满足风控反欺诈领域需求的，可拓展、高效的图神经
网络应用方案，从而帮助更好地识别欺诈用户。

本项目主要关于实现预测模型(**不限于图神经网络**)，进行节点异常检测任务，并验证模型精度。而本项目基于的数据集[DGraph](https://dgraph.xinye.com/introduction)，[DGraph](https://dgraph.xinye.com/introduction)
是大规模动态图数据集的集合，由真实金融场景中随着时间演变事件和标签构成。

### 1.1 实验目的

- 了解如何使用Pytorch进行神经网络训练
- 了解如何使用Pytorch-geometric等图网络深度学习库进行简单图神经网络设计(推荐使用GAT, GraphSAGE模型)。
- 了解如何利用MO平台进行模型性能评估。

### 1.2 预备知识
- 具备一定的深度学习理论知识，如卷积神经网络、损失函数、优化器，训练策略等。
- 了解并熟悉Pytorch计算框架。
- 学习Pytorch-geometric，请前往：https://pytorch-geometric.readthedocs.io/en/latest/
    
### 1.3实验环境
- numpy = 1.18.5  
- pytorch = 1.4.0  
- torch_geometric = 1.7.0  
- torch_scatter = 2.0.3  
- torch_sparse = 0.5.1  

## 2. 实验内容

### 2.1 数据集信息
DGraph-Fin 是一个由数百万个节点和边组成的有向无边权的动态图。它代表了Finvolution Group用户之间的社交网络，其中一个节点对应一个Finvolution 用户，从一个用户到另一个用户的边表示**该用户将另一个用户视为紧急联系人**。
下面是`位于dataset/DGraphFin目录`的DGraphFin数据集的描述:
```
x:  20维节点特征向量
y:  节点对应标签，一共包含四类。其中类1代表欺诈用户而类0代表正常用户(实验中需要进行预测的两类标签)，类2和类3则是背景用户，即无需预测其标签。
edge_index:  图数据边集,每条边的形式(id_a,id_b)，其中ids是x中的索引
edge_type: 共11种类型的边
edge_timestamp: 脱敏后的时间戳
train_mask, valid_mask, test_mask: 训练集，验证集和测试集掩码
```
本预测任务为识别欺诈用户的节点预测任务,只需要将欺诈用户（Class 1）从正常用户（Class 0）中区分出来。需要注意的是，其中测试集中样本对应的label**均被标记为-100**。

### 2.2 导入相关包

导入相应模块，设置数据集路径、设备等。

In [1]:
import torch
print(torch.__version__)    #查看cpu版本
print(torch.version.cuda)     #查看gpu版本

1.12.0
None


In [3]:
from utils import DGraphFin
from utils.utils import prepare_folder
from utils.evaluator import Evaluator

import torch
import torch.nn.functional as F
import torch.nn as nn

import torch_geometric.transforms as T

import numpy as np
from torch_geometric.data import Data
import os

#设置gpu设备
device = 0
device = f'cuda:{device}' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)


### 2.3 数据处理

在使用数据集训练网络前，首先需要对数据进行归一化等预处理，如下：

In [5]:
path='./datasets/632d74d4e2843a53167ee9a1-momodel/' #数据保存路径
save_dir='./results/' #模型保存路径
dataset_name='DGraph'
dataset = DGraphFin(root=path, name=dataset_name, transform=T.ToSparseTensor())

nlabels = dataset.num_classes
if dataset_name in ['DGraph']:
    nlabels = 2    #本实验中仅需预测类0和类1

data = dataset[0]
data.adj_t = data.adj_t.to_symmetric() #将有向图转化为无向图


if dataset_name in ['DGraph']:
    x = data.x
    x = (x - x.mean(0)) / x.std(0)
    data.x = x
if data.y.dim() == 2:
    data.y = data.y.squeeze(1)

split_idx = {'train': data.train_mask, 'valid': data.valid_mask, 'test': data.test_mask}  #划分训练集，验证集

train_idx = split_idx['train']
result_dir = prepare_folder(dataset_name,'mlp')

111
read_dgraph


Processing...
Done!


这里我们可以查看数据各部分维度

In [34]:
print(data)
print(data.x.shape)  #feature
print(data.x.shape)  #feature
print(data.y.shape)  #label
print(data.y)
print(data.train_mask)
print(data.test_mask)
print(data.valid_mask)
print(train_idx)

Data(x=[3700550, 20], edge_attr=[4300999], y=[3700550], train_mask=[857899], valid_mask=[183862], test_mask=[183840], adj_t=[3700550, 3700550, nnz=7994520])
torch.Size([3700550, 20])
torch.Size([3700550, 20])
torch.Size([3700550])
tensor([   2,    2, -100,  ...,    2,    0,    0])
tensor([2763073,  548373,  699951,  ...,  220416, 2843218, 2544039])
tensor([2683077, 2842796,  424346,  ..., 1345318,  458653, 2675150])
tensor([2825811, 1815662, 1229948,  ...,  417443, 2532587, 1678153])
tensor([2763073,  548373,  699951,  ...,  220416, 2843218, 2544039])


### 2.4 定义模型
这里我们使用简单的多层感知机作为例子：

In [7]:
class MLP(torch.nn.Module):
    def __init__(self
                 , in_channels
                 , hidden_channels
                 , out_channels
                 , num_layers
                 , dropout
                 , batchnorm=True):
        super(MLP, self).__init__()
        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.batchnorm = batchnorm
        if self.batchnorm:
            self.bns = torch.nn.ModuleList()
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            if self.batchnorm:
                self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for lin in self.lins:
            lin.reset_parameters()
        if self.batchnorm:
            for bn in self.bns:
                bn.reset_parameters()

    def forward(self, x):    
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            if self.batchnorm:
                x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return F.log_softmax(x, dim=-1)

配置后续训练、验证、推理用到的参数。可以调整以下超参以提高模型训练后的验证精度：

- `epochs`：在训练集上训练的代数；
- `lr`：学习率；
- `num_layers`：网络的层数；
- `hidden_channels`：隐藏层维数；
- `dropout`：dropout比例；
- `weight_decay`：正则化项的系数。

In [8]:
mlp_parameters = {
    'lr': 0.01
    , 'num_layers': 2
    , 'hidden_channels': 128
    , 'dropout': 0.0
    , 'batchnorm': False
    , 'weight_decay': 5e-7
                  }
epochs = 200
log_steps =10 # log记录周期

初始化模型，并使用**Area Under the Curve (AUC)** 作为模型评价指标来衡量模型的表现。AUC通过对ROC曲线下各部分的面积求和而得。

具体计算过程参见 https://github.com/scikit-learn/scikit-learn/blob/baf828ca1/sklearn/metrics/_ranking.py#L363

In [9]:
para_dict = mlp_parameters
model_para = mlp_parameters.copy()
model_para.pop('lr')
model_para.pop('weight_decay')
model = MLP(in_channels=data.x.size(-1), out_channels=nlabels, **model_para).to(device)
print(f'Model MLP initialized')


eval_metric = 'auc'  #使用AUC衡量指标
evaluator = Evaluator(eval_metric)

Model MLP initialized


### 2.5 训练

使用训练集中的节点用于训练模型，并使用验证集进行挑选模型。

In [10]:
def train(model, data, train_idx, optimizer):
     # data.y is labels of shape (N, ) 
    model.train()

    optimizer.zero_grad()
    
    out = model(data.x[train_idx])

    loss = F.nll_loss(out, data.y[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()


In [11]:
def test(model, data, split_idx, evaluator):
    # data.y is labels of shape (N, )
    with torch.no_grad():
        model.eval()

        losses, eval_results = dict(), dict()
        for key in ['train', 'valid']:
            node_id = split_idx[key]
            
            out = model(data.x[node_id])
            y_pred = out.exp()  # (N,num_classes)
            
            losses[key] = F.nll_loss(out, data.y[node_id]).item()
            eval_results[key] = evaluator.eval(data.y[node_id], y_pred)[eval_metric]

    return eval_results, losses, y_pred

In [12]:
print(sum(p.numel() for p in model.parameters()))  #模型总参数量

model.reset_parameters()
optimizer = torch.optim.Adam(model.parameters(), lr=para_dict['lr'], weight_decay=para_dict['weight_decay'])
best_valid = 0
min_valid_loss = 1e8

for epoch in range(1,epochs + 1):
    loss = train(model, data, train_idx, optimizer)
    eval_results, losses, out = test(model, data, split_idx, evaluator)
    train_eval, valid_eval = eval_results['train'], eval_results['valid']
    train_loss, valid_loss = losses['train'], losses['valid']

    if valid_loss < min_valid_loss:
        min_valid_loss = valid_loss
        torch.save(model.state_dict(), save_dir+'/model.pt') #将表现最好的模型保存

    if epoch % log_steps == 0:
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}, '
              f'Train: {100 * train_eval:.3f}, ' # 我们将AUC值乘上100，使其在0-100的区间内
              f'Valid: {100 * valid_eval:.3f} ')

2946
Epoch: 10, Loss: 0.0880, Train: 64.275, Valid: 64.651 
Epoch: 20, Loss: 0.0953, Train: 68.205, Valid: 67.830 
Epoch: 30, Loss: 0.0809, Train: 69.551, Valid: 68.857 
Epoch: 40, Loss: 0.0681, Train: 69.887, Valid: 69.603 
Epoch: 50, Loss: 0.0650, Train: 69.387, Valid: 69.230 
Epoch: 60, Loss: 0.0651, Train: 69.699, Valid: 69.473 
Epoch: 70, Loss: 0.0645, Train: 70.595, Valid: 70.271 
Epoch: 80, Loss: 0.0644, Train: 70.857, Valid: 70.468 
Epoch: 90, Loss: 0.0643, Train: 70.902, Valid: 70.514 
Epoch: 100, Loss: 0.0642, Train: 71.109, Valid: 70.673 
Epoch: 110, Loss: 0.0641, Train: 71.261, Valid: 70.793 
Epoch: 120, Loss: 0.0641, Train: 71.385, Valid: 70.864 
Epoch: 130, Loss: 0.0640, Train: 71.479, Valid: 70.931 
Epoch: 140, Loss: 0.0640, Train: 71.575, Valid: 70.991 
Epoch: 150, Loss: 0.0639, Train: 71.656, Valid: 71.043 
Epoch: 160, Loss: 0.0639, Train: 71.730, Valid: 71.091 
Epoch: 170, Loss: 0.0639, Train: 71.799, Valid: 71.132 
Epoch: 180, Loss: 0.0639, Train: 71.860, Valid: 71.1

### 2.6 模型预测

In [13]:
model.load_state_dict(torch.load(save_dir+'/model.pt')) #载入验证集上表现最好的模型
def predict(data,node_id):
    """
    加载模型和模型预测
    :param node_id: int, 需要进行预测节点的下标
    :return: tensor, 类0以及类1的概率, torch.size[1,2]
    """
    # -------------------------- 实现模型预测部分的代码 ---------------------------
    with torch.no_grad():
        model.eval()
        out = model(data.x[node_id])
        y_pred = out.exp()  # (N,num_classes)
        
    return y_pred

In [14]:
dic={0:"正常用户",1:"欺诈用户"}
node_idx = 0
y_pred = predict(data, node_idx)
print(y_pred)
print(f'节点 {node_idx} 预测对应的标签为:{torch.argmax(y_pred)}, 为{dic[torch.argmax(y_pred).item()]}。')

node_idx = 200
y_pred = predict(data, node_idx)
print(y_pred)
print(f'节点 {node_idx} 预测对应的标签为:{torch.argmax(y_pred)}, 为{dic[torch.argmax(y_pred).item()]}。')

tensor([0.9969, 0.0031])
节点 0 预测对应的标签为:0, 为正常用户。
tensor([0.9638, 0.0362])
节点 200 预测对应的标签为:0, 为正常用户。


In [38]:
results = []
for i in range(data.x.shape[0]):
        results.append(predict(data, i).unsqueeze_(0))
print(results)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [27]:
T1 = torch.tensor([1,2,5,4]).unsqueeze_(0)
T2 = torch.tensor([1,2,3,4]).unsqueeze_(0)
T3 = torch.tensor([2,2,3,4]).unsqueeze_(0)
a = []
a.append(T1)
a.append(T2)
a.append(T3)
a

[tensor([[1, 2, 5, 4]]), tensor([[1, 2, 3, 4]]), tensor([[2, 2, 3, 4]])]

In [39]:
results

[tensor([[0.9969, 0.0031]]),
 tensor([[0.9739, 0.0261]]),
 tensor([[0.9914, 0.0086]]),
 tensor([[0.9711, 0.0289]]),
 tensor([[0.9931, 0.0069]]),
 tensor([[0.9680, 0.0320]]),
 tensor([[0.9839, 0.0161]]),
 tensor([[0.9923, 0.0077]]),
 tensor([[0.9967, 0.0033]]),
 tensor([[0.9870, 0.0130]]),
 tensor([[0.9941, 0.0059]]),
 tensor([[9.9967e-01, 3.2816e-04]]),
 tensor([[0.9897, 0.0103]]),
 tensor([[0.9941, 0.0059]]),
 tensor([[0.9780, 0.0220]]),
 tensor([[0.9848, 0.0152]]),
 tensor([[0.9819, 0.0181]]),
 tensor([[0.9964, 0.0036]]),
 tensor([[0.9739, 0.0261]]),
 tensor([[0.9833, 0.0167]]),
 tensor([[0.9919, 0.0081]]),
 tensor([[0.9957, 0.0043]]),
 tensor([[0.9717, 0.0283]]),
 tensor([[0.9923, 0.0077]]),
 tensor([[0.9965, 0.0035]]),
 tensor([[0.9977, 0.0023]]),
 tensor([[0.9874, 0.0126]]),
 tensor([[0.9897, 0.0103]]),
 tensor([[0.9968, 0.0032]]),
 tensor([[0.9666, 0.0334]]),
 tensor([[0.9911, 0.0089]]),
 tensor([[0.9956, 0.0044]]),
 tensor([[0.9943, 0.0057]]),
 tensor([[0.9986, 0.0014]]),
 tenso

In [16]:
c = torch.cat(a,dim=0)
c

tensor([[1, 2, 5, 4],
        [1, 2, 3, 4],
        [2, 2, 3, 4]])

In [40]:
finall_result = torch.cat(results, dim = 0)

In [41]:
finall_result

tensor([[0.9969, 0.0031],
        [0.9739, 0.0261],
        [0.9914, 0.0086],
        ...,
        [0.9807, 0.0193],
        [0.9806, 0.0194],
        [0.9977, 0.0023]])

In [42]:
finall_result[0]

tensor([0.9969, 0.0031])

In [43]:
torch.save(finall_result, save_dir+'/results.pt')

## 3. 作业评分

**作业要求**：    
                         
1. 请加载你认为训练最佳的模型（不限于图神经网络)
2. 提交的作业包括【程序报告.pdf】和代码文件。

**注意：**
          
1. 在训练模型等过程中如果需要**保存数据、模型**等请写到 **results** 文件夹，如果采用 [离线任务](https://momodel.cn/docs/#/zh-cn/%E5%9C%A8GPU%E6%88%96CPU%E8%B5%84%E6%BA%90%E4%B8%8A%E8%AE%AD%E7%BB%83%E6%9C%BA%E5%99%A8%E5%AD%A6%E4%B9%A0%E6%A8%A1%E5%9E%8B) 请务必将模型保存在 **results** 文件夹下。
2. 训练出自己最好的模型后，先按照下列 cell 操作方式实现 NoteBook 加载模型测试；请测试通过在进行【系统测试】。
3. 点击左侧栏`提交作业`后点击`生成文件`则只需勾选 `predict()` 函数的cell，即【**模型预测代码答题区域**】的 cell。
4. 请导入必要的包和第三方库 (包括此文件中曾经导入过的)。
5. 请加载你认为训练最佳的模型，即请按要求填写**模型路径**。
6. `predict()`函数的输入和输出请不要改动。

===========================================  **模型预测代码答题区域**  =========================================== 

在下方的代码块中编写 **模型预测** 部分的代码，请勿在别的位置作答

In [17]:
## 生成 main.py 时请勾选此 cell
from utils import DGraphFin
from utils.evaluator import Evaluator
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch_geometric.transforms as T
from torch_geometric.data import Data
import numpy as np
import os

def predict(data,node_id):
    """
    加载模型和模型预测
    :param node_id: int, 需要进行预测节点的下标
    :return: tensor, 类0以及类1的概率, torch.size[1,2]
    """
    # 这里可以加载你的模型
    model = 
    model.load_state_dict(torch.load('./results/model.pt'))
    # 模型预测时，测试数据已经进行了归一化处理
    # -------------------------- 实现模型预测部分的代码 ---------------------------

    
    return y_pred